# What drives the price of a car?

![](images/kurt.jpeg)

### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

In [92]:
from itertools import product
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [93]:
df = pd.read_csv("data/vehicles.csv")
df.head()

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
0,7222695916,prescott,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az
1,7218891961,fayetteville,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar
2,7221797935,florida keys,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl
3,7222270760,worcester / central MA,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma
4,7210384030,greensboro,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc


In [94]:
test_results = {"MAE": [], "R":[]}

In [122]:
def run_drill(nan_thresh=5, price_thresh=5e5, mileage_thresh=1e6, commonality_thresh=10, components=25):
    df_lite = df.drop(columns=["id", "size", "paint_color"])  # retaining VIN until after the VIN duplicate check

    df_lite = df_lite[df_lite.notnull().sum(axis=1) >= nan_thresh] # dropping rows with fewer than n non-NaN values


    # Drop VIN duplicates then drop the VIN column since it has served its purpose
    df_lite = df_lite.drop_duplicates(subset='VIN', keep='first')
    df_lite = df_lite.drop(columns=["VIN"])


    # Drop exact duplicate rows (keep first occurrence)
    df_lite = df_lite.drop_duplicates()
    df_clean_imp = df_lite.copy()
    working_df = df_clean_imp.copy()

    # Mean/Mode Fill the working df if that is what is needed
    for col in working_df.columns:
        if working_df[col].dtype in ['float64', 'int64']:
            working_df[col] = working_df[col].fillna(working_df[col].mean())
        elif working_df[col].dtype == 'object':
            working_df[col] = working_df[col].fillna(working_df[col].mode()[0])

    # Prices and Odometer values approaching 1M should be examined/handled
    working_df = working_df[working_df['price'] <= price_thresh]
    working_df = working_df[(working_df['odometer'] <= mileage_thresh)]
    working_df.reset_index(drop=True, inplace=True)
    working_df.drop(columns=["region", "model"], inplace=True)

    american = [
        "gmc", "chevrolet", "ford", "jeep", "ram", "cadillac", "buick",
        "lincoln", "chrysler", "dodge", "pontiac", "mercury", "saturn",
        "tesla", "harley-davidson"]

    japanese = [
        "toyota", "nissan", "mazda", "honda", "lexus", "acura", "subaru",
        "mitsubishi", "infiniti", "datsun"]

    asian = [
        "hyundai", "kia"]

    european = [
        "volvo", "audi", "bmw", "mercedes-benz", "porsche", "fiat",
        "jaguar", "mini", "rover", "land rover", "alfa-romeo",
        "aston-martin", "ferrari", "volkswagen"]

    manufacturer_region = {}

    for brand in american:
        manufacturer_region[brand] = "american"
    for brand in japanese:
        manufacturer_region[brand] = "japanese"
    for brand in asian:
        manufacturer_region[brand] = "asian"
    for brand in european:
        manufacturer_region[brand] = "european"

    working_df['manufacturer_region'] = working_df['manufacturer'].map(manufacturer_region)
    working_df.drop(columns=['manufacturer'], inplace=True)

    # Droping these 469 records wont hurt
    working_df = working_df[working_df['cylinders'] != 'other']
    working_df['cylinders'] = working_df['cylinders'].str.extract('(\d+)').astype('int64')

    # COLLAPSE THE TYPE FEATURE TO THE TOP N MOST COMMON VALUES
    top_n = working_df['type'].value_counts().nlargest(commonality_thresh).index
    working_df['type'] = working_df['type'].apply(lambda x: x if x in top_n else 'other')
    working_df['type'] = working_df['type'].str.lower()

    # GROUP STATES BY NATIONAL REGION
    region = {
        "South": ["wv", "dc", "md", "va",
                  "ky", "tn", "nc", "ms",
                  "ar", "la", "al", "ga", "sc",
                  "fl", "de", "pr"],
        "Southwest": ["az", "nm", "ok", "tx"],
        "West": ["wa", "or", "ca", "nv", "id", "mt",
                 "wy", "ut", "co", "ak", "hi"],
        "Midwest": ["nd", "sd", "ne", "ks", "mn",
                    "ia", "mo", "wi", "il", "mi", "in",
                    "oh"],
        "Northeast": ["me", "vt", "ny", "nh", "ma",
                      "ri", "ct", "nj", "pa"]
    }

    def get_region(state):
        if state in region["South"]:
            return "south"
        elif state in region["Southwest"]:
            return "southwest"
        elif state in region["West"]:
            return "west"
        elif state in region["Midwest"]:
            return "midwest"
        elif state in region["Northeast"]:
            return "northeast"
        else:
            return "unknown"

    state_col = working_df["state"].tolist()
    region_col = []
    for st in state_col:
        region_col.append(get_region(st))
    working_df["title_region"] = region_col
    working_df.drop(columns=["state"], inplace=True)


    # ----- Columns -----
    ordinal_col = ['condition']
    ordinal_order = [['salvage', 'fair', 'good', 'excellent', 'like new', 'new']]

    nominal_cols = [
        'fuel', 'title_status', 'transmission', 'drive',
        'type', 'manufacturer_region', 'title_region'
    ]

    numeric_cols = ['price', 'year', 'odometer', 'cylinders']

    # ----- Transformers -----
    ordinal_enc = OrdinalEncoder(categories=ordinal_order)
    nominal_enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

    preprocessor = ColumnTransformer([
        ('ordinal', ordinal_enc, ordinal_col),
        ('nominal', nominal_enc, nominal_cols)
    ], remainder='passthrough')  # keep numeric features

    # ----- Final pipeline -----
    full_pipeline = Pipeline([
        ('encode', preprocessor)
    ])

    # ----- Transform the dataset -----
    working_df_transformed = full_pipeline.fit_transform(working_df)

    # Get feature names
    encoded_cols = list(full_pipeline.named_steps['encode'].transformers_[1][1].get_feature_names_out(nominal_cols))
    final_columns = ordinal_col + encoded_cols + numeric_cols

    working_df_encoded = pd.DataFrame(working_df_transformed, columns=final_columns)


    # Step 1: isolate features and target
    X = working_df_encoded.drop("price", axis=1)
    y = working_df_encoded["price"]

    # Step 2: split into train_temp (80%) and test (20%)
    X_train_temp, X_test, y_train_temp, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Step 3: split train_temp into train (75% of 80%) and dev (25% of 80%)
    X_train, X_dev, y_train, y_dev = train_test_split(
        X_train_temp, y_train_temp, test_size=0.25, random_state=42
    )

    numeric_features = ['year', 'odometer', 'cylinders']  # Possibly 'price' if you're transforming it


    # Instantiate the scaler
    scaler = StandardScaler()

    # Fit on training data ONLY
    scaler.fit(X_train[numeric_features])

    # Transform all sets
    X_train[numeric_features] = scaler.transform(X_train[numeric_features])
    X_dev[numeric_features] = scaler.transform(X_dev[numeric_features])
    X_test[numeric_features] = scaler.transform(X_test[numeric_features])


    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_dev_scaled = scaler.transform(X_dev)
    X_test_scaled = scaler.transform(X_test)

    pca = PCA(n_components=components, random_state=42)  # OR: n_components=20
    X_train_pca = pca.fit_transform(X_train_scaled)
    X_dev_pca = pca.transform(X_dev_scaled)
    X_test_pca = pca.transform(X_test_scaled)


    # Create transformer
    poly = PolynomialFeatures(degree=2, include_bias=False)

    # Fit + transform train only
    X_train_poly = poly.fit_transform(X_train_scaled)
    X_dev_poly = poly.transform(X_dev_scaled)
    X_test_poly = poly.transform(X_test_scaled)

    #Ridge Regressor
    model = Ridge(alpha=100)
    model.fit(X_train_poly, y_train)
    y_pred = model.predict(X_dev_poly)
    test_results["MAE"].append(mean_absolute_error(y_dev, y_pred))
    test_results["R"].append(r2_score(y_dev, y_pred))


    # RRandom Forest Regressor
    # model = RandomForestRegressor(
    #     n_estimators=100,
    #     max_depth=None,
    #     random_state=42,
    #     n_jobs=-1
    # )

    # Train
    model.fit(X_train, y_train)

    # Predict on dev set
    y_pred = model.predict(X_dev)

    # Evaluate
    test_results["MAE"].append(mean_absolute_error(y_dev, y_pred))
    test_results["R"].append(r2_score(y_dev, y_pred))

FOR SINGLE RUN TESTING

In [123]:
# run_drill()
# print(test_results)
# print(f"\nTime elapsed: {(toc() - start)/60:0.2f} minutes.")

THRESHOLD TEST BLOCK

In [120]:
nan_threshes = [4]
price_threshes = [1e6]
mileage_threshes = [1e6]
commonality_threshes = [9, 10]


for nan in nan_threshes:
    for price in price_threshes:
        for mileage in mileage_threshes:
            for commonality in commonality_threshes:
                run_drill(nan_thresh=nan, price_thresh=price,mileage_thresh=mileage, commonality_thresh=commonality)
print(test_results)

{'MAE': [7392.684839807434, 8161.553335541447, 7400.264632099564, 8164.332614132231], 'R': [0.34799487629364256, 0.2754790066206867, 0.347219571943413, 0.27567113994902703]}


THRESH TEST RESULTS

In [121]:
# Rebuild param grid
param_grid = list(product(nan_threshes, price_threshes, mileage_threshes, commonality_threshes))
param_df = pd.DataFrame(param_grid, columns=["nan_thresh", "price_thresh", "mileage_thresh", "commonality_thresh"])

# Rehydrate results
results_df = pd.DataFrame(test_results)
results_df = pd.concat([results_df, param_df], axis=1)

# Find best row by MAE or R²
best_mae_idx = results_df["MAE"].idxmin()
best_r2_idx = results_df["R"].idxmax()

print("🔹 Best by MAE:")
print(results_df.loc[best_mae_idx])

print("\n🔹 Best by R²:")
print(results_df.loc[best_r2_idx])

test_results["MAE"].clear()
test_results["R"].clear()


🔹 Best by MAE:
MAE                      7392.684840
R                           0.347995
nan_thresh                  4.000000
price_thresh          1000000.000000
mileage_thresh        1000000.000000
commonality_thresh          9.000000
Name: 0, dtype: float64

🔹 Best by R²:
MAE                      7392.684840
R                           0.347995
nan_thresh                  4.000000
price_thresh          1000000.000000
mileage_thresh        1000000.000000
commonality_thresh          9.000000
Name: 0, dtype: float64
